In [1]:
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import time
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from numpy.random import seed
seed(0)
tf.random.set_seed(0)
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.model_selection import cross_val_score, GridSearchCV
from keras.models import Sequential 
from keras.layers import Dense, Activation, Dropout 
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
import imblearn
from tensorflow import keras 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Permute
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Masking
#from keras.utils import plot_model
import sklearn
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Activation
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras import optimizers 
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV 

from scipy.stats import expon
from scipy.stats import uniform
from scipy.stats import gamma
from sklearn import metrics
from scipy.stats import reciprocal
from scipy.stats import randint
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [2]:
#Load peripheral preprocessed data
#rsp_df = joblib.load(open('/home/sowmya/QoE_D2/Dataset2/SpyderCodeDataset2/data/rsp_18pyhrvftr_d2.dat', 'rb'))
#rsp_data= np.array(rsp_df)
rsp_df = joblib.load(open('C:/Users/Sowmya/QoE/QoE_D2/Dataset2/SpyderCodeDataset2/data/rsp_40features.dat', 'rb'))
rsp_data= rsp_df.drop(['RRV_SampEn'], axis =1)
np.count_nonzero(np.isinf(rsp_data))

0

In [3]:
#Load peripheral preprocessed data
ecg_HRV, ratings = joblib.load(open('C:/Users/Sowmya/QoE/QoE_D2/Dataset2/SpyderCodeDataset2/data/ECG_18pyHRVfeatures_new.dat', 'rb'))
ecg_morpho, ratings = joblib.load(open('C:/Users/Sowmya/QoE/QoE_D2/Dataset2/SpyderCodeDataset2/data/ECG_morphofeatures.dat', 'rb'))

# Place the pyHRV and Morphological ff features DataFrames side by side
horizontal_stack = pd.concat([ecg_HRV, ecg_morpho, rsp_data], axis=1)
#drop columns with all NAN values
ff_df = horizontal_stack.dropna(axis=1, how='all')
ff_data= np.array(ff_df)


In [4]:
#Check if input contains zero and infinity or a value too large
np.count_nonzero(np.isnan(ff_data))
np.count_nonzero(np.isinf(ff_data))

#Indices containing inf values and convert them to NaN
indsinf = np.where(np.isinf(ff_data))
ff_data[indsinf] = 'NaN'

#Replace NaN values with mean of the column
col_mean = np.nanmean(ff_data, axis=0)    
inds = np.where(np.isnan(ff_data))
ff_data[inds] = np.take(col_mean, inds[1])
np.count_nonzero(np.isnan(ff_data))
np.count_nonzero(np.isinf(ff_data))

#ff= pd.DataFrame(data= ff_data, columns= ff_df.columns)

0

In [5]:
#load ratings from excel sheet as pandas dataframe
load_ratings = pd.read_excel ('C:/Users/Sowmya/QoE/QoE_D2/Dataset2/Ratings_Dataset2.xlsx', sheet_name='EI_ALL')
ratings1 = load_ratings.drop(['subject'], axis=1) #delete subject from dataframe
ff_label = ratings1.values #convert dataframe to numpy array

def data_binarizer_two(ratings):
	"""binarizes the data below and above the threshold"""
	binarized = []
	for rating in ratings:
		if rating <= 6:
			binarized.append(0)
		else:
			binarized.append(1)
	return binarized


from collections import Counter
y_ia = np.array(data_binarizer_two([el[0] for el in ff_label]))
y_aq = np.array(data_binarizer_two([el[1] for el in ff_label]))
y_iv = np.array(data_binarizer_two([el[2] for el in ff_label]))
y_il = np.array(data_binarizer_two([el[3] for el in ff_label]))
y_oq = np.array(data_binarizer_two([el[4] for el in ff_label]))
y_sa = np.array(data_binarizer_two([el[5] for el in ff_label]))

counter = Counter(y_il)
print(counter)


Counter({0: 317, 1: 223})


In [6]:
# Split the data into training/testing sets
#X_train_ff_ia, X_test_ff_ia, y_train_ff_ia, y_test_ff_ia = train_test_split(ff_data, y_ia, test_size=0.2, random_state=42, stratify = y_ia)
#X_train_ff_aq, X_test_ff_aq, y_train_ff_aq, y_test_ff_aq = train_test_split(ff_data, y_aq, test_size=0.2, random_state=42, stratify = y_aq)
#X_train_ff_iv, X_test_ff_iv, y_train_ff_iv, y_test_ff_iv = train_test_split(ff_data, y_iv, test_size=0.2, random_state=42, stratify = y_iv)
#X_train_ff_il, X_test_ff_il, y_train_ff_il, y_test_ff_il = train_test_split(ff_data, y_il, test_size=0.2, random_state=42, stratify = y_il)
X_train_ff_oq, X_test_ff_oq, y_train_ff_oq, y_test_ff_oq = train_test_split(ff_data, y_oq, test_size=0.2, random_state=42, stratify = y_oq)
X_train_ff_sa, X_test_ff_sa, y_train_ff_sa, y_test_ff_sa = train_test_split(ff_data, y_sa, test_size=0.2, random_state=42, stratify = y_sa)
  
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train_ff_oq= sc.fit_transform(X_train_ff_oq)
X_test_ff_oq = sc.transform(X_test_ff_oq)

'''
#principle component analysis
from sklearn.decomposition import PCA
pca = PCA(n_components=10)
X_train_ff_oq = pca.fit_transform(X_train_ff_oq)
X_test_ff_oq = pca.transform(X_test_ff_oq)

# number of components
n_pcs= pca.components_.shape[0]

# get the index of the most important feature on EACH component i.e. largest absolute value
# using LIST COMPREHENSION HERE
most_important = [np.abs(pca.components_[i]).argmax() for i in range(n_pcs)]
initial_feature_names = ff_df.columns
# get the names
most_important_names = [initial_feature_names[most_important[i]] for i in range(n_pcs)]

# using LIST COMPREHENSION HERE AGAIN
dic = {'PC{}'.format(i+1): most_important_names[i] for i in range(n_pcs)}

# build the dataframe
df = pd.DataFrame(sorted(dic.items()))
'''

smote = SMOTE(random_state=0)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_ff_oq, y_train_ff_oq)
X_test_resampled, y_test_resampled = X_test_ff_oq, y_test_ff_oq

from collections import Counter
# summarize observations by class labeL
counter = Counter(y_train_resampled)
print(counter)

ff_randsearch = SVC(gamma= 'auto', random_state = 0) 
ff_randsearch.fit(X_train_resampled, y_train_resampled) 
ff_randsearch_pred = ff_randsearch.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, ff_randsearch_pred))
print("SVM Accuracy:", round(metrics.accuracy_score(y_test_resampled, ff_randsearch_pred), 6))
print("SVM F1score:", round(metrics.f1_score(y_test_resampled, ff_randsearch_pred, average = 'weighted'), 6))


from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import LeaveOneOut
rskfcv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
loocv = LeaveOneOut()


#from sklearn.utils.fixes import loguniform
ff_svm_param_grid= {'C': expon(scale=100), 'gamma': expon(scale=.1), 'kernel': ['rbf'], 'class_weight':['balanced', None]}
#ff_svm_param_grid= {'C': reciprocal(1e0, 1e3), 'gamma': reciprocal(1e-3, 1e-2), 'kernel': ['rbf'], 'class_weight':['balanced', None]}

ff_randsearch = RandomizedSearchCV(SVC(decision_function_shape='ovo'),param_distributions = ff_svm_param_grid, n_iter = 100, n_jobs = -1, cv = 10, random_state = 0) 
ff_randsearch.fit(X_train_resampled, y_train_resampled) 
#ff_randsearch.fit(X_train_ff_il, y_train_ff_il) 
ff_randsearch_pred = ff_randsearch.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, ff_randsearch_pred))
print("SVM RandomCV Accuracy:", round(metrics.accuracy_score(y_test_resampled, ff_randsearch_pred), 6))
print("SVM RandomCV F1score:", round(metrics.f1_score(y_test_resampled, ff_randsearch_pred,average = 'weighted'), 6))
print(ff_randsearch.best_params_)

#List Hyperparameters that we want to tune.
leaf_size = list(range(1,50))
n_neighbors = list(range(1,30))
p=[1,2]

#Convert to dictionary
hyperparameters = dict(leaf_size=leaf_size, n_neighbors=n_neighbors, p=p)
#Create new KNN object
knn_2 = KNeighborsClassifier()
#Use GridSearch
clf = GridSearchCV(knn_2, hyperparameters, cv=10)
clf.fit(X_train_resampled, y_train_resampled) 
clf_pred = clf.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, clf_pred))
print("KNN Accuracy grid search:", round(metrics.accuracy_score(y_test_resampled, clf_pred), 6))
print("KNN F1score grid search:", round(metrics.f1_score(y_test_resampled, clf_pred, average = 'weighted'), 6))
print(clf.best_params_)

ff_knn_param_grid= {'n_neighbors': (1,2,3,10, 20,100)}
ff_randsearch = RandomizedSearchCV(KNeighborsClassifier(),param_distributions = ff_knn_param_grid, n_iter = 100, n_jobs = -1, cv = 10, random_state = 0) 
ff_randsearch.fit(X_train_resampled, y_train_resampled) 
ff_randsearch_pred = ff_randsearch.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, ff_randsearch_pred))
print("KNN Accuracy:", round(metrics.accuracy_score(y_test_resampled, ff_randsearch_pred), 6))
print("KNN F1score:", round(metrics.f1_score(y_test_resampled, ff_randsearch_pred, average = 'weighted'), 6))
print(ff_randsearch.best_params_)

ff_randsearch = RandomForestClassifier() 
ff_randsearch.fit(X_train_resampled, y_train_resampled) 
ff_randsearch_pred = ff_randsearch.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, ff_randsearch_pred))
print("RF Accuracy:", round(metrics.accuracy_score(y_test_resampled, ff_randsearch_pred), 6))
print("RF F1score:", round(metrics.f1_score(y_test_resampled, ff_randsearch_pred, average = 'weighted'), 6))

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

ff_randsearchRF = RandomizedSearchCV(RandomForestClassifier(),param_distributions = random_grid, n_iter = 100, n_jobs = -1, cv = 10, random_state = 0, verbose =2) 
ff_randsearchRF.fit(X_train_resampled, y_train_resampled) 
ff_randsearchRF_pred = ff_randsearchRF.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, ff_randsearchRF_pred))
print("RF Random Accuracy _PARAMS:", round(metrics.accuracy_score(y_test_resampled, ff_randsearchRF_pred), 6))
print("RF Random F1score _PARAMS:", round(metrics.f1_score(y_test_resampled, ff_randsearchRF_pred, average = 'weighted'), 6))


ff_randsearch = RandomForestClassifier(max_depth=10, random_state=0) 
ff_randsearch.fit(X_train_resampled, y_train_resampled) 
ff_randsearch_pred = ff_randsearch.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, ff_randsearch_pred))
print("RF Random Accuracy:", round(metrics.accuracy_score(y_test_resampled, ff_randsearch_pred), 6))
print("RF Random F1score:", round(metrics.f1_score(y_test_resampled, ff_randsearch_pred, average = 'weighted'), 6))

ff_knn_param_grid= {'n_neighbors': (1,2,3,10, 20,100)}
ff_randsearch = RandomizedSearchCV(KNeighborsClassifier(),param_distributions = ff_knn_param_grid, n_iter = 100, n_jobs = -1, cv = 10, random_state = 0) 
ff_randsearch.fit(X_train_resampled, y_train_resampled) 
ff_randsearch_pred = ff_randsearch.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, ff_randsearch_pred))
print("KNN Accuracy:", round(metrics.accuracy_score(y_test_resampled, ff_randsearch_pred), 6))
print("KNN F1score:", round(metrics.f1_score(y_test_resampled, ff_randsearch_pred, average = 'weighted'), 6))
print(ff_randsearch.best_params_)




Counter({0: 277, 1: 277})
              precision    recall  f1-score   support

           0       0.69      0.62      0.65        39
           1       0.79      0.84      0.82        69

    accuracy                           0.76       108
   macro avg       0.74      0.73      0.73       108
weighted avg       0.76      0.76      0.76       108

SVM Accuracy: 0.759259
SVM F1score: 0.756143
              precision    recall  f1-score   support

           0       0.62      0.26      0.36        39
           1       0.68      0.91      0.78        69

    accuracy                           0.68       108
   macro avg       0.65      0.58      0.57       108
weighted avg       0.66      0.68      0.63       108

SVM RandomCV Accuracy: 0.675926
SVM RandomCV F1score: 0.631313
{'C': 156.88961399691684, 'class_weight': None, 'gamma': 0.06538825690914733, 'kernel': 'rbf'}
              precision    recall  f1-score   support

           0       0.56      0.72      0.63        39
        

c:\Users\aquar\anaconda3\envs\TF2\lib\site-packages\sklearn\model_selection\_search.py:306: UserWarning: The total space of parameters 6 is smaller than n_iter=100. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


              precision    recall  f1-score   support

           0       0.59      0.69      0.64        39
           1       0.81      0.72      0.76        69

    accuracy                           0.71       108
   macro avg       0.70      0.71      0.70       108
weighted avg       0.73      0.71      0.72       108

KNN Accuracy: 0.712963
KNN F1score: 0.717113
{'n_neighbors': 1}
              precision    recall  f1-score   support

           0       0.64      0.64      0.64        39
           1       0.80      0.80      0.80        69

    accuracy                           0.74       108
   macro avg       0.72      0.72      0.72       108
weighted avg       0.74      0.74      0.74       108

RF Accuracy: 0.740741
RF F1score: 0.740741
{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4],

c:\Users\aquar\anaconda3\envs\TF2\lib\site-packages\sklearn\model_selection\_search.py:306: UserWarning: The total space of parameters 6 is smaller than n_iter=100. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


In [9]:
# Split the data into training/testing sets
#X_train_ff_ia, X_test_ff_ia, y_train_ff_ia, y_test_ff_ia = train_test_split(ff_data, y_ia, test_size=0.2, random_state=42, stratify = y_ia)
#X_train_ff_aq, X_test_ff_aq, y_train_ff_aq, y_test_ff_aq = train_test_split(ff_data, y_aq, test_size=0.2, random_state=42, stratify = y_aq)
#X_train_ff_iv, X_test_ff_iv, y_train_ff_iv, y_test_ff_iv = train_test_split(ff_data, y_iv, test_size=0.2, random_state=42, stratify = y_iv)
#X_train_ff_il, X_test_ff_il, y_train_ff_il, y_test_ff_il = train_test_split(ff_data, y_il, test_size=0.2, random_state=42, stratify = y_il)
X_train_ff_oq, X_test_ff_oq, y_train_ff_oq, y_test_ff_oq = train_test_split(ff_data, y_oq, test_size=0.2, random_state=42, stratify = y_oq)
X_train_ff_sa, X_test_ff_sa, y_train_ff_sa, y_test_ff_sa = train_test_split(ff_data, y_sa, test_size=0.2, random_state=42, stratify = y_sa)
  
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train_ff_oq= sc.fit_transform(X_train_ff_oq)
X_test_ff_oq = sc.transform(X_test_ff_oq)


#principle component analysis
from sklearn.decomposition import PCA
pca = PCA(n_components=5)
X_train_ff_oq = pca.fit_transform(X_train_ff_oq)
X_test_ff_oq = pca.transform(X_test_ff_oq)

# number of components
n_pcs= pca.components_.shape[0]

# get the index of the most important feature on EACH component i.e. largest absolute value
# using LIST COMPREHENSION HERE
most_important = [np.abs(pca.components_[i]).argmax() for i in range(n_pcs)]
initial_feature_names = ff_df.columns
# get the names
most_important_names = [initial_feature_names[most_important[i]] for i in range(n_pcs)]

# using LIST COMPREHENSION HERE AGAIN
dic = {'PC{}'.format(i+1): most_important_names[i] for i in range(n_pcs)}

# build the dataframe
df = pd.DataFrame(sorted(dic.items()))


smote = SMOTE(random_state=0)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_ff_oq, y_train_ff_oq)
X_test_resampled, y_test_resampled = X_test_ff_oq, y_test_ff_oq

from collections import Counter
# summarize observations by class labeL
counter = Counter(y_train_resampled)
print(counter)

ff_randsearch = SVC(gamma= 'auto', random_state = 0) 
ff_randsearch.fit(X_train_resampled, y_train_resampled) 
ff_randsearch_pred = ff_randsearch.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, ff_randsearch_pred))
print("SVM Accuracy:", round(metrics.accuracy_score(y_test_resampled, ff_randsearch_pred), 6))
print("SVM F1score:", round(metrics.f1_score(y_test_resampled, ff_randsearch_pred, average = 'weighted'), 6))


from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import LeaveOneOut
rskfcv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
loocv = LeaveOneOut()


#from sklearn.utils.fixes import loguniform
ff_svm_param_grid= {'C': expon(scale=100), 'gamma': expon(scale=.1), 'kernel': ['rbf'], 'class_weight':['balanced', None]}
#ff_svm_param_grid= {'C': reciprocal(1e0, 1e3), 'gamma': reciprocal(1e-3, 1e-2), 'kernel': ['rbf'], 'class_weight':['balanced', None]}

ff_randsearch = RandomizedSearchCV(SVC(decision_function_shape='ovo'),param_distributions = ff_svm_param_grid, n_iter = 100, n_jobs = -1, cv = 10, random_state = 0) 
ff_randsearch.fit(X_train_resampled, y_train_resampled) 
#ff_randsearch.fit(X_train_ff_il, y_train_ff_il) 
ff_randsearch_pred = ff_randsearch.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, ff_randsearch_pred))
print("SVM RandomCV Accuracy:", round(metrics.accuracy_score(y_test_resampled, ff_randsearch_pred), 6))
print("SVM RandomCV F1score:", round(metrics.f1_score(y_test_resampled, ff_randsearch_pred,average = 'weighted'), 6))
print(ff_randsearch.best_params_)

#List Hyperparameters that we want to tune.
leaf_size = list(range(1,50))
n_neighbors = list(range(1,30))
p=[1,2]

#Convert to dictionary
hyperparameters = dict(leaf_size=leaf_size, n_neighbors=n_neighbors, p=p)
#Create new KNN object
knn_2 = KNeighborsClassifier()
#Use GridSearch
clf = GridSearchCV(knn_2, hyperparameters, cv=10)
clf.fit(X_train_resampled, y_train_resampled) 
clf_pred = clf.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, clf_pred))
print("KNN Accuracy grid search:", round(metrics.accuracy_score(y_test_resampled, clf_pred), 6))
print("KNN F1score grid search:", round(metrics.f1_score(y_test_resampled, clf_pred, average = 'weighted'), 6))
print(clf.best_params_)

ff_knn_param_grid= {'n_neighbors': (1,2,3,10, 20,100)}
ff_randsearch = RandomizedSearchCV(KNeighborsClassifier(),param_distributions = ff_knn_param_grid, n_iter = 100, n_jobs = -1, cv = 10, random_state = 0) 
ff_randsearch.fit(X_train_resampled, y_train_resampled) 
ff_randsearch_pred = ff_randsearch.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, ff_randsearch_pred))
print("KNN Accuracy:", round(metrics.accuracy_score(y_test_resampled, ff_randsearch_pred), 6))
print("KNN F1score:", round(metrics.f1_score(y_test_resampled, ff_randsearch_pred, average = 'weighted'), 6))
print(ff_randsearch.best_params_)

ff_randsearch = RandomForestClassifier() 
ff_randsearch.fit(X_train_resampled, y_train_resampled) 
ff_randsearch_pred = ff_randsearch.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, ff_randsearch_pred))
print("RF Accuracy:", round(metrics.accuracy_score(y_test_resampled, ff_randsearch_pred), 6))
print("RF F1score:", round(metrics.f1_score(y_test_resampled, ff_randsearch_pred, average = 'weighted'), 6))

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

ff_randsearchRF = RandomizedSearchCV(RandomForestClassifier(),param_distributions = random_grid, n_iter = 100, n_jobs = -1, cv = 10, random_state = 0, verbose =2) 
ff_randsearchRF.fit(X_train_resampled, y_train_resampled) 
ff_randsearchRF_pred = ff_randsearchRF.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, ff_randsearchRF_pred))
print("RF Random Accuracy _PARAMS:", round(metrics.accuracy_score(y_test_resampled, ff_randsearchRF_pred), 6))
print("RF Random F1score _PARAMS:", round(metrics.f1_score(y_test_resampled, ff_randsearchRF_pred, average = 'weighted'), 6))


ff_randsearch = RandomForestClassifier(max_depth=10, random_state=0) 
ff_randsearch.fit(X_train_resampled, y_train_resampled) 
ff_randsearch_pred = ff_randsearch.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, ff_randsearch_pred))
print("RF Random Accuracy:", round(metrics.accuracy_score(y_test_resampled, ff_randsearch_pred), 6))
print("RF Random F1score:", round(metrics.f1_score(y_test_resampled, ff_randsearch_pred, average = 'weighted'), 6))



Counter({0: 277, 1: 277})
              precision    recall  f1-score   support

           0       0.62      0.77      0.69        39
           1       0.85      0.74      0.79        69

    accuracy                           0.75       108
   macro avg       0.74      0.75      0.74       108
weighted avg       0.77      0.75      0.75       108

SVM Accuracy: 0.75
SVM F1score: 0.75421
              precision    recall  f1-score   support

           0       0.58      0.46      0.51        39
           1       0.73      0.81      0.77        69

    accuracy                           0.69       108
   macro avg       0.65      0.64      0.64       108
weighted avg       0.67      0.69      0.68       108

SVM RandomCV Accuracy: 0.685185
SVM RandomCV F1score: 0.675821
{'C': 43.434565697036696, 'class_weight': 'balanced', 'gamma': 0.6765393734039725, 'kernel': 'rbf'}
              precision    recall  f1-score   support

           0       0.51      0.54      0.53        39
        

c:\Users\aquar\anaconda3\envs\TF2\lib\site-packages\sklearn\model_selection\_search.py:306: UserWarning: The total space of parameters 6 is smaller than n_iter=100. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


              precision    recall  f1-score   support

           0       0.51      0.54      0.53        39
           1       0.73      0.71      0.72        69

    accuracy                           0.65       108
   macro avg       0.62      0.62      0.62       108
weighted avg       0.65      0.65      0.65       108

KNN Accuracy: 0.648148
KNN F1score: 0.649959
{'n_neighbors': 1}
              precision    recall  f1-score   support

           0       0.65      0.62      0.63        39
           1       0.79      0.81      0.80        69

    accuracy                           0.74       108
   macro avg       0.72      0.71      0.72       108
weighted avg       0.74      0.74      0.74       108

RF Accuracy: 0.740741
RF F1score: 0.739181
{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4],

In [ ]:
# Split the data into training/testing sets
#X_train_ff_ia, X_test_ff_ia, y_train_ff_ia, y_test_ff_ia = train_test_split(ff_data, y_ia, test_size=0.2, random_state=42, stratify = y_ia)
#X_train_ff_aq, X_test_ff_aq, y_train_ff_aq, y_test_ff_aq = train_test_split(ff_data, y_aq, test_size=0.2, random_state=42, stratify = y_aq)
#X_train_ff_iv, X_test_ff_iv, y_train_ff_iv, y_test_ff_iv = train_test_split(ff_data, y_iv, test_size=0.2, random_state=42, stratify = y_iv)
#X_train_ff_il, X_test_ff_il, y_train_ff_il, y_test_ff_il = train_test_split(ff_data, y_il, test_size=0.2, random_state=42, stratify = y_il)
X_train_ff_oq, X_test_ff_oq, y_train_ff_oq, y_test_ff_oq = train_test_split(ff_data, y_oq, test_size=0.2, random_state=42, stratify = y_oq)
X_train_ff_sa, X_test_ff_sa, y_train_ff_sa, y_test_ff_sa = train_test_split(ff_data, y_sa, test_size=0.2, random_state=42, stratify = y_sa)
  
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train_ff_oq= sc.fit_transform(X_train_ff_oq)
X_test_ff_oq = sc.transform(X_test_ff_oq)


#principle component analysis
from sklearn.decomposition import PCA
pca = PCA(n_components=10)
X_train_ff_oq = pca.fit_transform(X_train_ff_oq)
X_test_ff_oq = pca.transform(X_test_ff_oq)

# number of components
n_pcs= pca.components_.shape[0]

# get the index of the most important feature on EACH component i.e. largest absolute value
# using LIST COMPREHENSION HERE
most_important = [np.abs(pca.components_[i]).argmax() for i in range(n_pcs)]
initial_feature_names = ff_df.columns
# get the names
most_important_names = [initial_feature_names[most_important[i]] for i in range(n_pcs)]

# using LIST COMPREHENSION HERE AGAIN
dic = {'PC{}'.format(i+1): most_important_names[i] for i in range(n_pcs)}

# build the dataframe
df = pd.DataFrame(sorted(dic.items()))


smote = SMOTE(random_state=0)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_ff_oq, y_train_ff_oq)
X_test_resampled, y_test_resampled = X_test_ff_oq, y_test_ff_oq

from collections import Counter
# summarize observations by class labeL
counter = Counter(y_train_resampled)
print(counter)

ff_randsearch = SVC(gamma= 'auto', random_state = 0) 
ff_randsearch.fit(X_train_resampled, y_train_resampled) 
ff_randsearch_pred = ff_randsearch.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, ff_randsearch_pred))
print("SVM Accuracy:", round(metrics.accuracy_score(y_test_resampled, ff_randsearch_pred), 6))
print("SVM F1score:", round(metrics.f1_score(y_test_resampled, ff_randsearch_pred, average = 'weighted'), 6))


from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import LeaveOneOut
rskfcv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
loocv = LeaveOneOut()


#from sklearn.utils.fixes import loguniform
ff_svm_param_grid= {'C': expon(scale=100), 'gamma': expon(scale=.1), 'kernel': ['rbf'], 'class_weight':['balanced', None]}
#ff_svm_param_grid= {'C': reciprocal(1e0, 1e3), 'gamma': reciprocal(1e-3, 1e-2), 'kernel': ['rbf'], 'class_weight':['balanced', None]}

ff_randsearch = RandomizedSearchCV(SVC(decision_function_shape='ovo'),param_distributions = ff_svm_param_grid, n_iter = 100, n_jobs = -1, cv = 10, random_state = 0) 
ff_randsearch.fit(X_train_resampled, y_train_resampled) 
#ff_randsearch.fit(X_train_ff_il, y_train_ff_il) 
ff_randsearch_pred = ff_randsearch.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, ff_randsearch_pred))
print("SVM RandomCV Accuracy:", round(metrics.accuracy_score(y_test_resampled, ff_randsearch_pred), 6))
print("SVM RandomCV F1score:", round(metrics.f1_score(y_test_resampled, ff_randsearch_pred,average = 'weighted'), 6))
print(ff_randsearch.best_params_)

#List Hyperparameters that we want to tune.
leaf_size = list(range(1,50))
n_neighbors = list(range(1,30))
p=[1,2]

#Convert to dictionary
hyperparameters = dict(leaf_size=leaf_size, n_neighbors=n_neighbors, p=p)
#Create new KNN object
knn_2 = KNeighborsClassifier()
#Use GridSearch
clf = GridSearchCV(knn_2, hyperparameters, cv=10)
clf.fit(X_train_resampled, y_train_resampled) 
clf_pred = clf.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, clf_pred))
print("KNN Accuracy grid search:", round(metrics.accuracy_score(y_test_resampled, clf_pred), 6))
print("KNN F1score grid search:", round(metrics.f1_score(y_test_resampled, clf_pred, average = 'weighted'), 6))
print(clf.best_params_)

ff_knn_param_grid= {'n_neighbors': (1,2,3,10, 20,100)}
ff_randsearch = RandomizedSearchCV(KNeighborsClassifier(),param_distributions = ff_knn_param_grid, n_iter = 100, n_jobs = -1, cv = 10, random_state = 0) 
ff_randsearch.fit(X_train_resampled, y_train_resampled) 
ff_randsearch_pred = ff_randsearch.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, ff_randsearch_pred))
print("KNN Accuracy:", round(metrics.accuracy_score(y_test_resampled, ff_randsearch_pred), 6))
print("KNN F1score:", round(metrics.f1_score(y_test_resampled, ff_randsearch_pred, average = 'weighted'), 6))
print(ff_randsearch.best_params_)

ff_randsearch = RandomForestClassifier() 
ff_randsearch.fit(X_train_resampled, y_train_resampled) 
ff_randsearch_pred = ff_randsearch.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, ff_randsearch_pred))
print("RF Accuracy:", round(metrics.accuracy_score(y_test_resampled, ff_randsearch_pred), 6))
print("RF F1score:", round(metrics.f1_score(y_test_resampled, ff_randsearch_pred, average = 'weighted'), 6))

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

ff_randsearchRF = RandomizedSearchCV(RandomForestClassifier(),param_distributions = random_grid, n_iter = 100, n_jobs = -1, cv = 10, random_state = 0, verbose =2) 
ff_randsearchRF.fit(X_train_resampled, y_train_resampled) 
ff_randsearchRF_pred = ff_randsearchRF.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, ff_randsearchRF_pred))
print("RF Random Accuracy _PARAMS:", round(metrics.accuracy_score(y_test_resampled, ff_randsearchRF_pred), 6))
print("RF Random F1score _PARAMS:", round(metrics.f1_score(y_test_resampled, ff_randsearchRF_pred, average = 'weighted'), 6))


ff_randsearch = RandomForestClassifier(max_depth=10, random_state=0) 
ff_randsearch.fit(X_train_resampled, y_train_resampled) 
ff_randsearch_pred = ff_randsearch.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, ff_randsearch_pred))
print("RF Random Accuracy:", round(metrics.accuracy_score(y_test_resampled, ff_randsearch_pred), 6))
print("RF Random F1score:", round(metrics.f1_score(y_test_resampled, ff_randsearch_pred, average = 'weighted'), 6))



Counter({0: 277, 1: 277})
              precision    recall  f1-score   support

           0       0.69      0.46      0.55        39
           1       0.74      0.88      0.81        69

    accuracy                           0.73       108
   macro avg       0.72      0.67      0.68       108
weighted avg       0.73      0.73      0.72       108

SVM Accuracy: 0.731481
SVM F1score: 0.716188
              precision    recall  f1-score   support

           0       0.68      0.38      0.49        39
           1       0.72      0.90      0.80        69

    accuracy                           0.71       108
   macro avg       0.70      0.64      0.65       108
weighted avg       0.71      0.71      0.69       108

SVM RandomCV Accuracy: 0.712963
SVM RandomCV F1score: 0.688707
{'C': 85.8392964354515, 'class_weight': 'balanced', 'gamma': 0.22637335586430757, 'kernel': 'rbf'}
              precision    recall  f1-score   support

           0       0.61      0.69      0.65        39
    

c:\Users\aquar\anaconda3\envs\TF2\lib\site-packages\sklearn\model_selection\_search.py:306: UserWarning: The total space of parameters 6 is smaller than n_iter=100. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


              precision    recall  f1-score   support

           0       0.61      0.69      0.65        39
           1       0.81      0.75      0.78        69

    accuracy                           0.73       108
   macro avg       0.71      0.72      0.72       108
weighted avg       0.74      0.73      0.73       108

KNN Accuracy: 0.731481
KNN F1score: 0.734522
{'n_neighbors': 1}
              precision    recall  f1-score   support

           0       0.55      0.54      0.55        39
           1       0.74      0.75      0.75        69

    accuracy                           0.68       108
   macro avg       0.65      0.65      0.65       108
weighted avg       0.67      0.68      0.67       108

RF Accuracy: 0.675926
RF F1score: 0.674987
{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4],

c:\Users\aquar\anaconda3\envs\TF2\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


              precision    recall  f1-score   support

           0       0.64      0.54      0.58        39
           1       0.76      0.83      0.79        69

    accuracy                           0.72       108
   macro avg       0.70      0.68      0.69       108
weighted avg       0.72      0.72      0.72       108

RF Random Accuracy _PARAMS: 0.722222
RF Random F1score _PARAMS: 0.716435
              precision    recall  f1-score   support

           0       0.58      0.54      0.56        39
           1       0.75      0.78      0.77        69

    accuracy                           0.69       108
   macro avg       0.67      0.66      0.66       108
weighted avg       0.69      0.69      0.69       108

RF Random Accuracy: 0.694444
RF Random F1score: 0.691584


In [8]:
# Split the data into training/testing sets
#X_train_ff_ia, X_test_ff_ia, y_train_ff_ia, y_test_ff_ia = train_test_split(ff_data, y_ia, test_size=0.2, random_state=42, stratify = y_ia)
#X_train_ff_aq, X_test_ff_aq, y_train_ff_aq, y_test_ff_aq = train_test_split(ff_data, y_aq, test_size=0.2, random_state=42, stratify = y_aq)
#X_train_ff_iv, X_test_ff_iv, y_train_ff_iv, y_test_ff_iv = train_test_split(ff_data, y_iv, test_size=0.2, random_state=42, stratify = y_iv)
#X_train_ff_il, X_test_ff_il, y_train_ff_il, y_test_ff_il = train_test_split(ff_data, y_il, test_size=0.2, random_state=42, stratify = y_il)
X_train_ff_oq, X_test_ff_oq, y_train_ff_oq, y_test_ff_oq = train_test_split(ff_data, y_oq, test_size=0.2, random_state=42, stratify = y_oq)
X_train_ff_sa, X_test_ff_sa, y_train_ff_sa, y_test_ff_sa = train_test_split(ff_data, y_sa, test_size=0.2, random_state=42, stratify = y_sa)
  
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train_ff_oq= sc.fit_transform(X_train_ff_oq)
X_test_ff_oq = sc.transform(X_test_ff_oq)


#principle component analysis
from sklearn.decomposition import PCA
pca = PCA(n_components=20)
X_train_ff_oq = pca.fit_transform(X_train_ff_oq)
X_test_ff_oq = pca.transform(X_test_ff_oq)

# number of components
n_pcs= pca.components_.shape[0]

# get the index of the most important feature on EACH component i.e. largest absolute value
# using LIST COMPREHENSION HERE
most_important = [np.abs(pca.components_[i]).argmax() for i in range(n_pcs)]
initial_feature_names = ff_df.columns
# get the names
most_important_names = [initial_feature_names[most_important[i]] for i in range(n_pcs)]

# using LIST COMPREHENSION HERE AGAIN
dic = {'PC{}'.format(i+1): most_important_names[i] for i in range(n_pcs)}

# build the dataframe
df = pd.DataFrame(sorted(dic.items()))


smote = SMOTE(random_state=0)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_ff_oq, y_train_ff_oq)
X_test_resampled, y_test_resampled = X_test_ff_oq, y_test_ff_oq

from collections import Counter
# summarize observations by class labeL
counter = Counter(y_train_resampled)
print(counter)

ff_randsearch = SVC(gamma= 'auto', random_state = 0) 
ff_randsearch.fit(X_train_resampled, y_train_resampled) 
ff_randsearch_pred = ff_randsearch.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, ff_randsearch_pred))
print("SVM Accuracy:", round(metrics.accuracy_score(y_test_resampled, ff_randsearch_pred), 6))
print("SVM F1score:", round(metrics.f1_score(y_test_resampled, ff_randsearch_pred, average = 'weighted'), 6))


from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import LeaveOneOut
rskfcv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
loocv = LeaveOneOut()


#from sklearn.utils.fixes import loguniform
ff_svm_param_grid= {'C': expon(scale=100), 'gamma': expon(scale=.1), 'kernel': ['rbf'], 'class_weight':['balanced', None]}
#ff_svm_param_grid= {'C': reciprocal(1e0, 1e3), 'gamma': reciprocal(1e-3, 1e-2), 'kernel': ['rbf'], 'class_weight':['balanced', None]}

ff_randsearch = RandomizedSearchCV(SVC(decision_function_shape='ovo'),param_distributions = ff_svm_param_grid, n_iter = 100, n_jobs = -1, cv = 10, random_state = 0) 
ff_randsearch.fit(X_train_resampled, y_train_resampled) 
#ff_randsearch.fit(X_train_ff_il, y_train_ff_il) 
ff_randsearch_pred = ff_randsearch.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, ff_randsearch_pred))
print("SVM RandomCV Accuracy:", round(metrics.accuracy_score(y_test_resampled, ff_randsearch_pred), 6))
print("SVM RandomCV F1score:", round(metrics.f1_score(y_test_resampled, ff_randsearch_pred,average = 'weighted'), 6))
print(ff_randsearch.best_params_)

#List Hyperparameters that we want to tune.
leaf_size = list(range(1,50))
n_neighbors = list(range(1,30))
p=[1,2]

#Convert to dictionary
hyperparameters = dict(leaf_size=leaf_size, n_neighbors=n_neighbors, p=p)
#Create new KNN object
knn_2 = KNeighborsClassifier()
#Use GridSearch
clf = GridSearchCV(knn_2, hyperparameters, cv=10)
clf.fit(X_train_resampled, y_train_resampled) 
clf_pred = clf.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, clf_pred))
print("KNN Accuracy grid search:", round(metrics.accuracy_score(y_test_resampled, clf_pred), 6))
print("KNN F1score grid search:", round(metrics.f1_score(y_test_resampled, clf_pred, average = 'weighted'), 6))
print(clf.best_params_)

ff_knn_param_grid= {'n_neighbors': (1,2,3,10, 20,100)}
ff_randsearch = RandomizedSearchCV(KNeighborsClassifier(),param_distributions = ff_knn_param_grid, n_iter = 100, n_jobs = -1, cv = 10, random_state = 0) 
ff_randsearch.fit(X_train_resampled, y_train_resampled) 
ff_randsearch_pred = ff_randsearch.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, ff_randsearch_pred))
print("KNN Accuracy:", round(metrics.accuracy_score(y_test_resampled, ff_randsearch_pred), 6))
print("KNN F1score:", round(metrics.f1_score(y_test_resampled, ff_randsearch_pred, average = 'weighted'), 6))
print(ff_randsearch.best_params_)

ff_randsearch = RandomForestClassifier() 
ff_randsearch.fit(X_train_resampled, y_train_resampled) 
ff_randsearch_pred = ff_randsearch.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, ff_randsearch_pred))
print("RF Accuracy:", round(metrics.accuracy_score(y_test_resampled, ff_randsearch_pred), 6))
print("RF F1score:", round(metrics.f1_score(y_test_resampled, ff_randsearch_pred, average = 'weighted'), 6))

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

ff_randsearchRF = RandomizedSearchCV(RandomForestClassifier(),param_distributions = random_grid, n_iter = 100, n_jobs = -1, cv = 10, random_state = 0, verbose =2) 
ff_randsearchRF.fit(X_train_resampled, y_train_resampled) 
ff_randsearchRF_pred = ff_randsearchRF.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, ff_randsearchRF_pred))
print("RF Random Accuracy _PARAMS:", round(metrics.accuracy_score(y_test_resampled, ff_randsearchRF_pred), 6))
print("RF Random F1score _PARAMS:", round(metrics.f1_score(y_test_resampled, ff_randsearchRF_pred, average = 'weighted'), 6))


ff_randsearch = RandomForestClassifier(max_depth=10, random_state=0) 
ff_randsearch.fit(X_train_resampled, y_train_resampled) 
ff_randsearch_pred = ff_randsearch.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, ff_randsearch_pred))
print("RF Random Accuracy:", round(metrics.accuracy_score(y_test_resampled, ff_randsearch_pred), 6))
print("RF Random F1score:", round(metrics.f1_score(y_test_resampled, ff_randsearch_pred, average = 'weighted'), 6))



Counter({0: 277, 1: 277})
              precision    recall  f1-score   support

           0       0.69      0.51      0.59        39
           1       0.76      0.87      0.81        69

    accuracy                           0.74       108
   macro avg       0.72      0.69      0.70       108
weighted avg       0.73      0.74      0.73       108

SVM Accuracy: 0.740741
SVM F1score: 0.730436
              precision    recall  f1-score   support

           0       0.71      0.26      0.38        39
           1       0.69      0.94      0.80        69

    accuracy                           0.69       108
   macro avg       0.70      0.60      0.59       108
weighted avg       0.70      0.69      0.65       108

SVM RandomCV Accuracy: 0.694444
SVM RandomCV F1score: 0.645812
{'C': 193.65772332668504, 'class_weight': None, 'gamma': 0.1447794086464392, 'kernel': 'rbf'}
              precision    recall  f1-score   support

           0       0.62      0.64      0.63        39
         

c:\Users\aquar\anaconda3\envs\TF2\lib\site-packages\sklearn\model_selection\_search.py:306: UserWarning: The total space of parameters 6 is smaller than n_iter=100. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


              precision    recall  f1-score   support

           0       0.62      0.64      0.63        39
           1       0.79      0.78      0.79        69

    accuracy                           0.73       108
   macro avg       0.71      0.71      0.71       108
weighted avg       0.73      0.73      0.73       108

KNN Accuracy: 0.731481
KNN F1score: 0.732201
{'n_neighbors': 1}
              precision    recall  f1-score   support

           0       0.77      0.44      0.56        39
           1       0.74      0.93      0.83        69

    accuracy                           0.75       108
   macro avg       0.76      0.68      0.69       108
weighted avg       0.75      0.75      0.73       108

RF Accuracy: 0.75
RF F1score: 0.728874
{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bo

c:\Users\aquar\anaconda3\envs\TF2\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


              precision    recall  f1-score   support

           0       0.76      0.49      0.59        39
           1       0.76      0.91      0.83        69

    accuracy                           0.76       108
   macro avg       0.76      0.70      0.71       108
weighted avg       0.76      0.76      0.74       108

RF Random Accuracy _PARAMS: 0.759259
RF Random F1score _PARAMS: 0.744015
              precision    recall  f1-score   support

           0       0.71      0.51      0.60        39
           1       0.76      0.88      0.82        69

    accuracy                           0.75       108
   macro avg       0.74      0.70      0.71       108
weighted avg       0.75      0.75      0.74       108

RF Random Accuracy: 0.75
RF Random F1score: 0.738706


In [6]:
# Split the data into training/testing sets
#X_train_ff_ia, X_test_ff_ia, y_train_ff_ia, y_test_ff_ia = train_test_split(ff_data, y_ia, test_size=0.2, random_state=42, stratify = y_ia)
#X_train_ff_aq, X_test_ff_aq, y_train_ff_aq, y_test_ff_aq = train_test_split(ff_data, y_aq, test_size=0.2, random_state=42, stratify = y_aq)
#X_train_ff_iv, X_test_ff_iv, y_train_ff_iv, y_test_ff_iv = train_test_split(ff_data, y_iv, test_size=0.2, random_state=42, stratify = y_iv)
#X_train_ff_il, X_test_ff_il, y_train_ff_il, y_test_ff_il = train_test_split(ff_data, y_il, test_size=0.2, random_state=42, stratify = y_il)
X_train_ff_oq, X_test_ff_oq, y_train_ff_oq, y_test_ff_oq = train_test_split(ff_data, y_oq, test_size=0.2, random_state=42, stratify = y_oq)
X_train_ff_sa, X_test_ff_sa, y_train_ff_sa, y_test_ff_sa = train_test_split(ff_data, y_sa, test_size=0.2, random_state=42, stratify = y_sa)
  
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train_ff_oq= sc.fit_transform(X_train_ff_oq)
X_test_ff_oq = sc.transform(X_test_ff_oq)


#principle component analysis
from sklearn.decomposition import PCA
pca = PCA(n_components=30)
X_train_ff_oq = pca.fit_transform(X_train_ff_oq)
X_test_ff_oq = pca.transform(X_test_ff_oq)

# number of components
n_pcs= pca.components_.shape[0]

# get the index of the most important feature on EACH component i.e. largest absolute value
# using LIST COMPREHENSION HERE
most_important = [np.abs(pca.components_[i]).argmax() for i in range(n_pcs)]
initial_feature_names = ff_df.columns
# get the names
most_important_names = [initial_feature_names[most_important[i]] for i in range(n_pcs)]

# using LIST COMPREHENSION HERE AGAIN
dic = {'PC{}'.format(i+1): most_important_names[i] for i in range(n_pcs)}

# build the dataframe
df = pd.DataFrame(sorted(dic.items()))


smote = SMOTE(random_state=0)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_ff_oq, y_train_ff_oq)
X_test_resampled, y_test_resampled = X_test_ff_oq, y_test_ff_oq

from collections import Counter
# summarize observations by class labeL
counter = Counter(y_train_resampled)
print(counter)

ff_randsearch = SVC(gamma= 'auto', random_state = 0) 
ff_randsearch.fit(X_train_resampled, y_train_resampled) 
ff_randsearch_pred = ff_randsearch.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, ff_randsearch_pred))
print("SVM Accuracy:", round(metrics.accuracy_score(y_test_resampled, ff_randsearch_pred), 6))
print("SVM F1score:", round(metrics.f1_score(y_test_resampled, ff_randsearch_pred, average = 'weighted'), 6))


from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import LeaveOneOut
rskfcv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
loocv = LeaveOneOut()


#from sklearn.utils.fixes import loguniform
ff_svm_param_grid= {'C': expon(scale=100), 'gamma': expon(scale=.1), 'kernel': ['rbf'], 'class_weight':['balanced', None]}
#ff_svm_param_grid= {'C': reciprocal(1e0, 1e3), 'gamma': reciprocal(1e-3, 1e-2), 'kernel': ['rbf'], 'class_weight':['balanced', None]}

ff_randsearch = RandomizedSearchCV(SVC(decision_function_shape='ovo'),param_distributions = ff_svm_param_grid, n_iter = 100, n_jobs = -1, cv = 10, random_state = 0) 
ff_randsearch.fit(X_train_resampled, y_train_resampled) 
#ff_randsearch.fit(X_train_ff_il, y_train_ff_il) 
ff_randsearch_pred = ff_randsearch.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, ff_randsearch_pred))
print("SVM RandomCV Accuracy:", round(metrics.accuracy_score(y_test_resampled, ff_randsearch_pred), 6))
print("SVM RandomCV F1score:", round(metrics.f1_score(y_test_resampled, ff_randsearch_pred,average = 'weighted'), 6))
print(ff_randsearch.best_params_)

#List Hyperparameters that we want to tune.
leaf_size = list(range(1,50))
n_neighbors = list(range(1,30))
p=[1,2]

#Convert to dictionary
hyperparameters = dict(leaf_size=leaf_size, n_neighbors=n_neighbors, p=p)
#Create new KNN object
knn_2 = KNeighborsClassifier()
#Use GridSearch
clf = GridSearchCV(knn_2, hyperparameters, cv=10)
clf.fit(X_train_resampled, y_train_resampled) 
clf_pred = clf.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, clf_pred))
print("KNN Accuracy grid search:", round(metrics.accuracy_score(y_test_resampled, clf_pred), 6))
print("KNN F1score grid search:", round(metrics.f1_score(y_test_resampled, clf_pred, average = 'weighted'), 6))
print(clf.best_params_)

ff_knn_param_grid= {'n_neighbors': (1,2,3,10, 20,100)}
ff_randsearch = RandomizedSearchCV(KNeighborsClassifier(),param_distributions = ff_knn_param_grid, n_iter = 100, n_jobs = -1, cv = 10, random_state = 0) 
ff_randsearch.fit(X_train_resampled, y_train_resampled) 
ff_randsearch_pred = ff_randsearch.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, ff_randsearch_pred))
print("KNN Accuracy:", round(metrics.accuracy_score(y_test_resampled, ff_randsearch_pred), 6))
print("KNN F1score:", round(metrics.f1_score(y_test_resampled, ff_randsearch_pred, average = 'weighted'), 6))
print(ff_randsearch.best_params_)

ff_randsearch = RandomForestClassifier() 
ff_randsearch.fit(X_train_resampled, y_train_resampled) 
ff_randsearch_pred = ff_randsearch.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, ff_randsearch_pred))
print("RF Accuracy:", round(metrics.accuracy_score(y_test_resampled, ff_randsearch_pred), 6))
print("RF F1score:", round(metrics.f1_score(y_test_resampled, ff_randsearch_pred, average = 'weighted'), 6))

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

ff_randsearchRF = RandomizedSearchCV(RandomForestClassifier(),param_distributions = random_grid, n_iter = 100, n_jobs = -1, cv = 10, random_state = 0, verbose =2) 
ff_randsearchRF.fit(X_train_resampled, y_train_resampled) 
ff_randsearchRF_pred = ff_randsearchRF.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, ff_randsearchRF_pred))
print("RF Random Accuracy _PARAMS:", round(metrics.accuracy_score(y_test_resampled, ff_randsearchRF_pred), 6))
print("RF Random F1score _PARAMS:", round(metrics.f1_score(y_test_resampled, ff_randsearchRF_pred, average = 'weighted'), 6))


ff_randsearch = RandomForestClassifier(max_depth=10, random_state=0) 
ff_randsearch.fit(X_train_resampled, y_train_resampled) 
ff_randsearch_pred = ff_randsearch.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, ff_randsearch_pred))
print("RF Random Accuracy:", round(metrics.accuracy_score(y_test_resampled, ff_randsearch_pred), 6))
print("RF Random F1score:", round(metrics.f1_score(y_test_resampled, ff_randsearch_pred, average = 'weighted'), 6))



Counter({0: 277, 1: 277})
              precision    recall  f1-score   support

           0       0.68      0.49      0.57        39
           1       0.75      0.87      0.81        69

    accuracy                           0.73       108
   macro avg       0.71      0.68      0.69       108
weighted avg       0.72      0.73      0.72       108

SVM Accuracy: 0.731481
SVM F1score: 0.719351
              precision    recall  f1-score   support

           0       0.61      0.28      0.39        39
           1       0.69      0.90      0.78        69

    accuracy                           0.68       108
   macro avg       0.65      0.59      0.58       108
weighted avg       0.66      0.68      0.64       108

SVM RandomCV Accuracy: 0.675926
SVM RandomCV F1score: 0.637629
{'C': 20.2350436780055, 'class_weight': 'balanced', 'gamma': 0.0804797390243771, 'kernel': 'rbf'}
              precision    recall  f1-score   support

           0       0.49      0.51      0.50        39
     

c:\Users\aquar\anaconda3\envs\TF2\lib\site-packages\sklearn\model_selection\_search.py:306: UserWarning: The total space of parameters 6 is smaller than n_iter=100. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


              precision    recall  f1-score   support

           0       0.51      0.54      0.53        39
           1       0.73      0.71      0.72        69

    accuracy                           0.65       108
   macro avg       0.62      0.62      0.62       108
weighted avg       0.65      0.65      0.65       108

KNN Accuracy: 0.648148
KNN F1score: 0.649959
{'n_neighbors': 1}
              precision    recall  f1-score   support

           0       0.71      0.51      0.60        39
           1       0.76      0.88      0.82        69

    accuracy                           0.75       108
   macro avg       0.74      0.70      0.71       108
weighted avg       0.75      0.75      0.74       108

RF Accuracy: 0.75
RF F1score: 0.738706
{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bo

In [ ]:
# Split the data into training/testing sets
#X_train_ff_ia, X_test_ff_ia, y_train_ff_ia, y_test_ff_ia = train_test_split(ff_data, y_ia, test_size=0.2, random_state=42, stratify = y_ia)
#X_train_ff_aq, X_test_ff_aq, y_train_ff_aq, y_test_ff_aq = train_test_split(ff_data, y_aq, test_size=0.2, random_state=42, stratify = y_aq)
#X_train_ff_iv, X_test_ff_iv, y_train_ff_iv, y_test_ff_iv = train_test_split(ff_data, y_iv, test_size=0.2, random_state=42, stratify = y_iv)
#X_train_ff_il, X_test_ff_il, y_train_ff_il, y_test_ff_il = train_test_split(ff_data, y_il, test_size=0.2, random_state=42, stratify = y_il)
X_train_ff_oq, X_test_ff_oq, y_train_ff_oq, y_test_ff_oq = train_test_split(ff_data, y_oq, test_size=0.2, random_state=42, stratify = y_oq)
X_train_ff_sa, X_test_ff_sa, y_train_ff_sa, y_test_ff_sa = train_test_split(ff_data, y_sa, test_size=0.2, random_state=42, stratify = y_sa)
  
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train_ff_oq= sc.fit_transform(X_train_ff_oq)
X_test_ff_oq = sc.transform(X_test_ff_oq)


#principle component analysis
from sklearn.decomposition import PCA
pca = PCA(n_components=40)
X_train_ff_oq = pca.fit_transform(X_train_ff_oq)
X_test_ff_oq = pca.transform(X_test_ff_oq)

# number of components
n_pcs= pca.components_.shape[0]

# get the index of the most important feature on EACH component i.e. largest absolute value
# using LIST COMPREHENSION HERE
most_important = [np.abs(pca.components_[i]).argmax() for i in range(n_pcs)]
initial_feature_names = ff_df.columns
# get the names
most_important_names = [initial_feature_names[most_important[i]] for i in range(n_pcs)]

# using LIST COMPREHENSION HERE AGAIN
dic = {'PC{}'.format(i+1): most_important_names[i] for i in range(n_pcs)}

# build the dataframe
df = pd.DataFrame(sorted(dic.items()))


smote = SMOTE(random_state=0)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_ff_oq, y_train_ff_oq)
X_test_resampled, y_test_resampled = X_test_ff_oq, y_test_ff_oq

from collections import Counter
# summarize observations by class labeL
counter = Counter(y_train_resampled)
print(counter)

ff_randsearch = SVC(gamma= 'auto', random_state = 0) 
ff_randsearch.fit(X_train_resampled, y_train_resampled) 
ff_randsearch_pred = ff_randsearch.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, ff_randsearch_pred))
print("SVM Accuracy:", round(metrics.accuracy_score(y_test_resampled, ff_randsearch_pred), 6))
print("SVM F1score:", round(metrics.f1_score(y_test_resampled, ff_randsearch_pred, average = 'weighted'), 6))


from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import LeaveOneOut
rskfcv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
loocv = LeaveOneOut()


#from sklearn.utils.fixes import loguniform
ff_svm_param_grid= {'C': expon(scale=100), 'gamma': expon(scale=.1), 'kernel': ['rbf'], 'class_weight':['balanced', None]}
#ff_svm_param_grid= {'C': reciprocal(1e0, 1e3), 'gamma': reciprocal(1e-3, 1e-2), 'kernel': ['rbf'], 'class_weight':['balanced', None]}

ff_randsearch = RandomizedSearchCV(SVC(decision_function_shape='ovo'),param_distributions = ff_svm_param_grid, n_iter = 100, n_jobs = -1, cv = 10, random_state = 0) 
ff_randsearch.fit(X_train_resampled, y_train_resampled) 
#ff_randsearch.fit(X_train_ff_il, y_train_ff_il) 
ff_randsearch_pred = ff_randsearch.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, ff_randsearch_pred))
print("SVM RandomCV Accuracy:", round(metrics.accuracy_score(y_test_resampled, ff_randsearch_pred), 6))
print("SVM RandomCV F1score:", round(metrics.f1_score(y_test_resampled, ff_randsearch_pred,average = 'weighted'), 6))
print(ff_randsearch.best_params_)

#List Hyperparameters that we want to tune.
leaf_size = list(range(1,50))
n_neighbors = list(range(1,30))
p=[1,2]

#Convert to dictionary
hyperparameters = dict(leaf_size=leaf_size, n_neighbors=n_neighbors, p=p)
#Create new KNN object
knn_2 = KNeighborsClassifier()
#Use GridSearch
clf = GridSearchCV(knn_2, hyperparameters, cv=10)
clf.fit(X_train_resampled, y_train_resampled) 
clf_pred = clf.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, clf_pred))
print("KNN Accuracy grid search:", round(metrics.accuracy_score(y_test_resampled, clf_pred), 6))
print("KNN F1score grid search:", round(metrics.f1_score(y_test_resampled, clf_pred, average = 'weighted'), 6))
print(clf.best_params_)

ff_knn_param_grid= {'n_neighbors': (1,2,3,10, 20,100)}
ff_randsearch = RandomizedSearchCV(KNeighborsClassifier(),param_distributions = ff_knn_param_grid, n_iter = 100, n_jobs = -1, cv = 10, random_state = 0) 
ff_randsearch.fit(X_train_resampled, y_train_resampled) 
ff_randsearch_pred = ff_randsearch.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, ff_randsearch_pred))
print("KNN Accuracy:", round(metrics.accuracy_score(y_test_resampled, ff_randsearch_pred), 6))
print("KNN F1score:", round(metrics.f1_score(y_test_resampled, ff_randsearch_pred, average = 'weighted'), 6))
print(ff_randsearch.best_params_)

ff_randsearch = RandomForestClassifier() 
ff_randsearch.fit(X_train_resampled, y_train_resampled) 
ff_randsearch_pred = ff_randsearch.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, ff_randsearch_pred))
print("RF Accuracy:", round(metrics.accuracy_score(y_test_resampled, ff_randsearch_pred), 6))
print("RF F1score:", round(metrics.f1_score(y_test_resampled, ff_randsearch_pred, average = 'weighted'), 6))

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

ff_randsearchRF = RandomizedSearchCV(RandomForestClassifier(),param_distributions = random_grid, n_iter = 100, n_jobs = -1, cv = 10, random_state = 0, verbose =2) 
ff_randsearchRF.fit(X_train_resampled, y_train_resampled) 
ff_randsearchRF_pred = ff_randsearchRF.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, ff_randsearchRF_pred))
print("RF Random Accuracy _PARAMS:", round(metrics.accuracy_score(y_test_resampled, ff_randsearchRF_pred), 6))
print("RF Random F1score _PARAMS:", round(metrics.f1_score(y_test_resampled, ff_randsearchRF_pred, average = 'weighted'), 6))


ff_randsearch = RandomForestClassifier(max_depth=10, random_state=0) 
ff_randsearch.fit(X_train_resampled, y_train_resampled) 
ff_randsearch_pred = ff_randsearch.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, ff_randsearch_pred))
print("RF Random Accuracy:", round(metrics.accuracy_score(y_test_resampled, ff_randsearch_pred), 6))
print("RF Random F1score:", round(metrics.f1_score(y_test_resampled, ff_randsearch_pred, average = 'weighted'), 6))



Counter({0: 277, 1: 277})
              precision    recall  f1-score   support

           0       0.69      0.51      0.59        39
           1       0.76      0.87      0.81        69

    accuracy                           0.74       108
   macro avg       0.72      0.69      0.70       108
weighted avg       0.73      0.74      0.73       108

SVM Accuracy: 0.740741
SVM F1score: 0.730436
              precision    recall  f1-score   support

           0       0.67      0.21      0.31        39
           1       0.68      0.94      0.79        69

    accuracy                           0.68       108
   macro avg       0.67      0.57      0.55       108
weighted avg       0.67      0.68      0.62       108

SVM RandomCV Accuracy: 0.675926
SVM RandomCV F1score: 0.616657
{'C': 20.2350436780055, 'class_weight': 'balanced', 'gamma': 0.0804797390243771, 'kernel': 'rbf'}
              precision    recall  f1-score   support

           0       0.51      0.56      0.54        39
     

c:\Users\aquar\anaconda3\envs\TF2\lib\site-packages\sklearn\model_selection\_search.py:306: UserWarning: The total space of parameters 6 is smaller than n_iter=100. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


              precision    recall  f1-score   support

           0       0.59      0.62      0.60        39
           1       0.78      0.75      0.76        69

    accuracy                           0.70       108
   macro avg       0.68      0.68      0.68       108
weighted avg       0.71      0.70      0.71       108

KNN Accuracy: 0.703704
KNN F1score: 0.705229
{'n_neighbors': 1}
              precision    recall  f1-score   support

           0       0.70      0.54      0.61        39
           1       0.77      0.87      0.82        69

    accuracy                           0.75       108
   macro avg       0.73      0.70      0.71       108
weighted avg       0.74      0.75      0.74       108

RF Accuracy: 0.75
RF F1score: 0.741349
{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bo

c:\Users\aquar\anaconda3\envs\TF2\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


              precision    recall  f1-score   support

           0       0.67      0.41      0.51        39
           1       0.73      0.88      0.80        69

    accuracy                           0.71       108
   macro avg       0.70      0.65      0.65       108
weighted avg       0.70      0.71      0.69       108

RF Random Accuracy _PARAMS: 0.712963
RF Random F1score _PARAMS: 0.692862
              precision    recall  f1-score   support

           0       0.62      0.46      0.53        39
           1       0.73      0.84      0.78        69

    accuracy                           0.70       108
   macro avg       0.68      0.65      0.66       108
weighted avg       0.69      0.70      0.69       108

RF Random Accuracy: 0.703704
RF Random F1score: 0.691927


In [10]:
# Split the data into training/testing sets
#X_train_ff_ia, X_test_ff_ia, y_train_ff_ia, y_test_ff_ia = train_test_split(ff_data, y_ia, test_size=0.2, random_state=42, stratify = y_ia)
#X_train_ff_aq, X_test_ff_aq, y_train_ff_aq, y_test_ff_aq = train_test_split(ff_data, y_aq, test_size=0.2, random_state=42, stratify = y_aq)
#X_train_ff_iv, X_test_ff_iv, y_train_ff_iv, y_test_ff_iv = train_test_split(ff_data, y_iv, test_size=0.2, random_state=42, stratify = y_iv)
#X_train_ff_il, X_test_ff_il, y_train_ff_il, y_test_ff_il = train_test_split(ff_data, y_il, test_size=0.2, random_state=42, stratify = y_il)
X_train_ff_oq, X_test_ff_oq, y_train_ff_oq, y_test_ff_oq = train_test_split(ff_data, y_oq, test_size=0.2, random_state=42, stratify = y_oq)
X_train_ff_sa, X_test_ff_sa, y_train_ff_sa, y_test_ff_sa = train_test_split(ff_data, y_sa, test_size=0.2, random_state=42, stratify = y_sa)
  
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train_ff_oq= sc.fit_transform(X_train_ff_oq)
X_test_ff_oq = sc.transform(X_test_ff_oq)


#principle component analysis
from sklearn.decomposition import PCA
pca = PCA(n_components=50)
X_train_ff_oq = pca.fit_transform(X_train_ff_oq)
X_test_ff_oq = pca.transform(X_test_ff_oq)

# number of components
n_pcs= pca.components_.shape[0]

# get the index of the most important feature on EACH component i.e. largest absolute value
# using LIST COMPREHENSION HERE
most_important = [np.abs(pca.components_[i]).argmax() for i in range(n_pcs)]
initial_feature_names = ff_df.columns
# get the names
most_important_names = [initial_feature_names[most_important[i]] for i in range(n_pcs)]

# using LIST COMPREHENSION HERE AGAIN
dic = {'PC{}'.format(i+1): most_important_names[i] for i in range(n_pcs)}

# build the dataframe
df = pd.DataFrame(sorted(dic.items()))


smote = SMOTE(random_state=0)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_ff_oq, y_train_ff_oq)
X_test_resampled, y_test_resampled = X_test_ff_oq, y_test_ff_oq

from collections import Counter
# summarize observations by class labeL
counter = Counter(y_train_resampled)
print(counter)

ff_randsearch = SVC(gamma= 'auto', random_state = 0) 
ff_randsearch.fit(X_train_resampled, y_train_resampled) 
ff_randsearch_pred = ff_randsearch.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, ff_randsearch_pred))
print("SVM Accuracy:", round(metrics.accuracy_score(y_test_resampled, ff_randsearch_pred), 6))
print("SVM F1score:", round(metrics.f1_score(y_test_resampled, ff_randsearch_pred, average = 'weighted'), 6))


from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import LeaveOneOut
rskfcv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
loocv = LeaveOneOut()


#from sklearn.utils.fixes import loguniform
ff_svm_param_grid= {'C': expon(scale=100), 'gamma': expon(scale=.1), 'kernel': ['rbf'], 'class_weight':['balanced', None]}
#ff_svm_param_grid= {'C': reciprocal(1e0, 1e3), 'gamma': reciprocal(1e-3, 1e-2), 'kernel': ['rbf'], 'class_weight':['balanced', None]}

ff_randsearch = RandomizedSearchCV(SVC(decision_function_shape='ovo'),param_distributions = ff_svm_param_grid, n_iter = 100, n_jobs = -1, cv = 10, random_state = 0) 
ff_randsearch.fit(X_train_resampled, y_train_resampled) 
#ff_randsearch.fit(X_train_ff_il, y_train_ff_il) 
ff_randsearch_pred = ff_randsearch.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, ff_randsearch_pred))
print("SVM RandomCV Accuracy:", round(metrics.accuracy_score(y_test_resampled, ff_randsearch_pred), 6))
print("SVM RandomCV F1score:", round(metrics.f1_score(y_test_resampled, ff_randsearch_pred,average = 'weighted'), 6))
print(ff_randsearch.best_params_)

#List Hyperparameters that we want to tune.
leaf_size = list(range(1,50))
n_neighbors = list(range(1,30))
p=[1,2]

#Convert to dictionary
hyperparameters = dict(leaf_size=leaf_size, n_neighbors=n_neighbors, p=p)
#Create new KNN object
knn_2 = KNeighborsClassifier()
#Use GridSearch
clf = GridSearchCV(knn_2, hyperparameters, cv=10)
clf.fit(X_train_resampled, y_train_resampled) 
clf_pred = clf.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, clf_pred))
print("KNN Accuracy grid search:", round(metrics.accuracy_score(y_test_resampled, clf_pred), 6))
print("KNN F1score grid search:", round(metrics.f1_score(y_test_resampled, clf_pred, average = 'weighted'), 6))
print(clf.best_params_)

ff_knn_param_grid= {'n_neighbors': (1,2,3,10, 20,100)}
ff_randsearch = RandomizedSearchCV(KNeighborsClassifier(),param_distributions = ff_knn_param_grid, n_iter = 100, n_jobs = -1, cv = 10, random_state = 0) 
ff_randsearch.fit(X_train_resampled, y_train_resampled) 
ff_randsearch_pred = ff_randsearch.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, ff_randsearch_pred))
print("KNN Accuracy:", round(metrics.accuracy_score(y_test_resampled, ff_randsearch_pred), 6))
print("KNN F1score:", round(metrics.f1_score(y_test_resampled, ff_randsearch_pred, average = 'weighted'), 6))
print(ff_randsearch.best_params_)

ff_randsearch = RandomForestClassifier() 
ff_randsearch.fit(X_train_resampled, y_train_resampled) 
ff_randsearch_pred = ff_randsearch.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, ff_randsearch_pred))
print("RF Accuracy:", round(metrics.accuracy_score(y_test_resampled, ff_randsearch_pred), 6))
print("RF F1score:", round(metrics.f1_score(y_test_resampled, ff_randsearch_pred, average = 'weighted'), 6))

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

ff_randsearchRF = RandomizedSearchCV(RandomForestClassifier(),param_distributions = random_grid, n_iter = 100, n_jobs = -1, cv = 10, random_state = 0, verbose =2) 
ff_randsearchRF.fit(X_train_resampled, y_train_resampled) 
ff_randsearchRF_pred = ff_randsearchRF.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, ff_randsearchRF_pred))
print("RF Random Accuracy _PARAMS:", round(metrics.accuracy_score(y_test_resampled, ff_randsearchRF_pred), 6))
print("RF Random F1score _PARAMS:", round(metrics.f1_score(y_test_resampled, ff_randsearchRF_pred, average = 'weighted'), 6))


ff_randsearch = RandomForestClassifier(max_depth=10, random_state=0) 
ff_randsearch.fit(X_train_resampled, y_train_resampled) 
ff_randsearch_pred = ff_randsearch.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, ff_randsearch_pred))
print("RF Random Accuracy:", round(metrics.accuracy_score(y_test_resampled, ff_randsearch_pred), 6))
print("RF Random F1score:", round(metrics.f1_score(y_test_resampled, ff_randsearch_pred, average = 'weighted'), 6))



Counter({0: 277, 1: 277})
              precision    recall  f1-score   support

           0       0.64      0.54      0.58        39
           1       0.76      0.83      0.79        69

    accuracy                           0.72       108
   macro avg       0.70      0.68      0.69       108
weighted avg       0.72      0.72      0.72       108

SVM Accuracy: 0.722222
SVM F1score: 0.716435
              precision    recall  f1-score   support

           0       0.60      0.31      0.41        39
           1       0.69      0.88      0.78        69

    accuracy                           0.68       108
   macro avg       0.65      0.60      0.59       108
weighted avg       0.66      0.68      0.64       108

SVM RandomCV Accuracy: 0.675926
SVM RandomCV F1score: 0.643354
{'C': 6.961767177320046, 'class_weight': None, 'gamma': 0.06512307303628603, 'kernel': 'rbf'}
              precision    recall  f1-score   support

           0       0.52      0.62      0.56        39
         

c:\Users\aquar\anaconda3\envs\TF2\lib\site-packages\sklearn\model_selection\_search.py:306: UserWarning: The total space of parameters 6 is smaller than n_iter=100. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


              precision    recall  f1-score   support

           0       0.57      0.64      0.60        39
           1       0.78      0.72      0.75        69

    accuracy                           0.69       108
   macro avg       0.67      0.68      0.68       108
weighted avg       0.70      0.69      0.70       108

KNN Accuracy: 0.694444
KNN F1score: 0.697904
{'n_neighbors': 1}
              precision    recall  f1-score   support

           0       0.66      0.54      0.59        39
           1       0.76      0.84      0.80        69

    accuracy                           0.73       108
   macro avg       0.71      0.69      0.70       108
weighted avg       0.72      0.73      0.72       108

RF Accuracy: 0.731481
RF F1score: 0.724726
{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4],

c:\Users\aquar\anaconda3\envs\TF2\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


              precision    recall  f1-score   support

           0       0.64      0.46      0.54        39
           1       0.74      0.86      0.79        69

    accuracy                           0.71       108
   macro avg       0.69      0.66      0.66       108
weighted avg       0.70      0.71      0.70       108

RF Random Accuracy _PARAMS: 0.712963
RF Random F1score _PARAMS: 0.699996
              precision    recall  f1-score   support

           0       0.66      0.49      0.56        39
           1       0.75      0.86      0.80        69

    accuracy                           0.72       108
   macro avg       0.70      0.67      0.68       108
weighted avg       0.71      0.72      0.71       108

RF Random Accuracy: 0.722222
RF Random F1score: 0.711182
